In [1]:
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk
import sqlite3

# Cài đặt cơ sở dữ liệu
def init_db():
    conn = sqlite3.connect("sanpham.db")
    cursor = conn.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS sanpham (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        ten TEXT NOT NULL,
        gia REAL NOT NULL
    )
    """)
    conn.commit()
    conn.close()

# Các hàm xử lý cơ sở dữ liệu
def them_san_pham(ten, gia):
    conn = sqlite3.connect("sanpham.db")
    cursor = conn.cursor()
    cursor.execute("INSERT INTO sanpham (ten, gia) VALUES (?, ?)", (ten, gia))
    conn.commit()
    conn.close()

def tim_kiem_san_pham(ten):
    conn = sqlite3.connect("sanpham.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM sanpham WHERE ten LIKE ?", ('%' + ten + '%',))
    ket_qua = cursor.fetchall()
    conn.close()
    return ket_qua

def cap_nhat_san_pham(id_san_pham, ten, gia):
    conn = sqlite3.connect("sanpham.db")
    cursor = conn.cursor()
    cursor.execute("UPDATE sanpham SET ten = ?, gia = ? WHERE id = ?", (ten, gia, id_san_pham))
    conn.commit()
    conn.close()

def xoa_san_pham(id_san_pham):
    conn = sqlite3.connect("sanpham.db")
    cursor = conn.cursor()
    cursor.execute("DELETE FROM sanpham WHERE id = ?", (id_san_pham,))
    conn.commit()
    conn.close()

def lay_tat_ca_san_pham():
    conn = sqlite3.connect("sanpham.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM sanpham")
    ket_qua = cursor.fetchall()
    conn.close()
    return ket_qua

# Cài đặt giao diện GUI
def main():
    def cap_nhat_danh_sach():
        for item in tree.get_children():
            tree.delete(item)
        for row in lay_tat_ca_san_pham():
            tree.insert("", "end", values=row)

    def su_kien_them():
        ten = ten_entry.get()
        gia = gia_entry.get()
        if ten and gia:
            try:
                them_san_pham(ten, float(gia))
                cap_nhat_danh_sach()
                ten_entry.delete(0, tk.END)
                gia_entry.delete(0, tk.END)
            except ValueError:
                messagebox.showerror("Lỗi", "Giá phải là một số.")
        else:
            messagebox.showwarning("Cảnh báo", "Vui lòng điền đầy đủ thông tin.")

    def su_kien_tim_kiem():
        ten = ten_entry.get()
        ket_qua = tim_kiem_san_pham(ten)
        for item in tree.get_children():
            tree.delete(item)
        for row in ket_qua:
            tree.insert("", "end", values=row)

    def su_kien_cap_nhat():
        item_da_chon = tree.selection()
        if item_da_chon:
            item = tree.item(item_da_chon)
            id_san_pham = item['values'][0]
            ten = ten_entry.get()
            gia = gia_entry.get()
            if ten and gia:
                try:
                    cap_nhat_san_pham(id_san_pham, ten, float(gia))
                    cap_nhat_danh_sach()
                except ValueError:
                    messagebox.showerror("Lỗi", "Giá phải là một số.")
            else:
                messagebox.showwarning("Cảnh báo", "Vui lòng điền đầy đủ thông tin.")
        else:
            messagebox.showwarning("Cảnh báo", "Vui lòng chọn sản phẩm để cập nhật.")

    def su_kien_xoa():
        item_da_chon = tree.selection()
        if item_da_chon:
            item = tree.item(item_da_chon)
            id_san_pham = item['values'][0]
            xoa_san_pham(id_san_pham)
            cap_nhat_danh_sach()
        else:
            messagebox.showwarning("Cảnh báo", "Vui lòng chọn sản phẩm để xóa.")

    # Cửa sổ chính
    root = tk.Tk()
    root.title("Quản Lý Sản Phẩm")

    # Các widget
    frame = tk.Frame(root)
    frame.pack(pady=10)

    tk.Label(frame, text="Tên:").grid(row=0, column=0, padx=5, pady=5)
    ten_entry = tk.Entry(frame)
    ten_entry.grid(row=0, column=1, padx=5, pady=5)

    tk.Label(frame, text="Giá:").grid(row=0, column=2, padx=5, pady=5)
    gia_entry = tk.Entry(frame)
    gia_entry.grid(row=0, column=3, padx=5, pady=5)

    them_button = tk.Button(frame, text="Thêm", command=su_kien_them)
    them_button.grid(row=0, column=4, padx=5, pady=5)

    tim_kiem_button = tk.Button(frame, text="Tìm Kiếm", command=su_kien_tim_kiem)
    tim_kiem_button.grid(row=0, column=5, padx=5, pady=5)

    cap_nhat_button = tk.Button(frame, text="Cập Nhật", command=su_kien_cap_nhat)
    cap_nhat_button.grid(row=0, column=6, padx=5, pady=5)

    xoa_button = tk.Button(frame, text="Xóa", command=su_kien_xoa)
    xoa_button.grid(row=0, column=7, padx=5, pady=5)

    # Treeview hiển thị danh sách sản phẩm
    columns = ("ID", "Tên", "Giá")
    tree = ttk.Treeview(root, columns=columns, show="headings")
    tree.heading("ID", text="ID")
    tree.heading("Tên", text="Tên")
    tree.heading("Giá", text="Giá")
    tree.pack(pady=10)

    # Cập nhật dữ liệu ban đầu
    cap_nhat_danh_sach()

    root.mainloop()

if __name__ == "__main__":
    init_db()
    main()
